In [4]:
!cd ./Gemini-Hackathon

In [ ]:
!pwd

❯ git config --global user.email "shng2025@gmail.com"
❯ git config --global user.name "Ice-Citron"

/data


In [ ]:
# Installation of Python-3.11
conda install -n base -c conda-forge python=3.11 -y

In [ ]:
conda create -n skyhammer311 -c conda-forge python=3.11 -y
conda activate skyhammer311
which python

python -m pip install -U pip
pip install -r requirements.txt


huggingface-cli login
wandb login

In [ ]:
# 1. Create Environment
conda create -n skyhammer_serve -c conda-forge python=3.11 -y

# 2. Activate
conda activate skyhammer_serve
which python

# 3. Install vLLM (Clean install)
pip install -r requirements-serve.txt

In [ ]:
# Just press Enter for defaults, mostly. 
# Key choices: 
# - Multi-GPU: YES
# - Num machines: 1
# - Num processes: 8
# - Mixed precision: bf16 (Since you have L40s)
accelerate config

export PYTORCH_ALLOC_CONF=expandable_segments:True
accelerate launch --multi_gpu --num_processes=8 src/train/train_ddp.py

In [ ]:
# 1. Set Memory Management Flag     # 2. Run on Single GPU (Safe Mode)
export PYTORCH_ALLOC_CONF=expandable_segments:True
CUDA_VISIBLE_DEVICES=0 python src/train/train_ddp.py

In [ ]:
conda deactivate
conda remove --name skyhammer311 --all -y

# VLLM SERVE

In [ ]:
vllm serve Qwen/Qwen2.5-Coder-32B-Instruct-AWQ \
  --tensor-parallel-size 2 \
  --enable-lora \
  --lora-modules skyhammer=shng2025/SkyHammer-32B-v1 \
  --port 8000 \
  --api-key skyhammer

In [ ]:
vllm serve Qwen/Qwen3-VL-32B-Instruct \
  --tensor-parallel-size 2 \
  --max-model-len 32768 \
  --port 8000 \
  --trust-remote-code \
  --api-key skyhammer

In [ ]:
huggingface-cli download unsloth/GLM-4.7-GGUF \
  --include "Q4_K_M/*" \
  --local-dir models/ \
  --local-dir-use-symlinks False


# pip install "llama-cpp-python[server]"
CMAKE_ARGS="-DGGML_CUDA=on" pip install --upgrade --force-reinstall --no-cache-dir llama-cpp-python[server]

python -m llama_cpp.server \
  --model models/Q4_K_M/GLM-4.7-Q4_K_M-00001-of-00005.gguf \
  --n_gpu_layers -1 \
  --n_ctx 32768 \
  --port 8000 \
  --api_key skyhammer




# Unsloth Trial

### FUCKING STUPID READ THE DOCUMENTATIONS NEXT TIME BAHAHAHAHAHA

In [ ]:
# 1. Clean up potential conflicts
pip uninstall -y transformers unsloth unsloth_zoo tokenizers

# 2. Install the Official Mistral Tokenizer Backend (The key from your docs!)
pip install --upgrade mistral-common>=1.5.5

# 3. Install Standard Training Support
pip install bitsandbytes accelerate peft sentencepiece protobuf datasets hf_transfer

# 4. Install the Experimental Transformers (Required for 'ministral3' model support)
pip install "git+https://github.com/huggingface/transformers.git@bf3f0ae70d0e902efab4b8517fce88f6697636ce"

# 5. Install Tokenizers (Updated to match the experimental transformers)
pip install --upgrade "tokenizers>=0.21.0"

# 6. Install Unsloth & TRL (No dependencies to avoid downgrading transformers)
pip install --no-deps "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git" unsloth_zoo trl==0.22.2


########
pip uninstall -y torchao
# pip install deepspeed==0.11.2

In [ ]:
CUDA_VISIBLE_DEVICES=0 python src/train/train_zero3_devstral.py


# Set timeouts to 2 hours (7200 seconds) to prevent "False Alarm" kills
export TORCH_NCCL_HEARTBEAT_TIMEOUT_SEC=7200
export NCCL_TIMEOUT=7200
export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
accelerate launch src/train/train_zero3_devstral.py

accelerate launch src/train/train_fsdp_24b.py